# Ejemplo CodeGen decoder

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# cargamos el tokenizador y el modelo de CodeGen, algo antiguo pero sirve 
# para mostrar el por qué hay que usar SFT
tokenizer = AutoTokenizer.from_pretrained("Salesforce/codegen-350M-mono")
# lo cargamos en bfloat16 para ahorrar memoria
model = AutoModelForCausalLM.from_pretrained("Salesforce/codegen-350M-mono", 
                                            trust_remote_code=True, revision="main", 
                                            device_map="auto",
                                            dtype=torch.bfloat16)

# Usar el modelo de manera naive

In [ ]:
from transformers import set_seed
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
set_seed(42)

# esto no funciona, problemas de la forma de entender el modelo
PROMPT = """### Instruction:
{instruction}

### Response:
{response}"""

# generamos el input
text = PROMPT.format(
    instruction="compute the mean of a numpy array",
    response=""
)
# lo tokenizamos
tokenized = tokenizer(text, return_tensors="pt")
input_ids = tokenized.input_ids
attention_mask = tokenized.attention_mask
# generamos la continuación
generated_ids = model.generate(input_ids.cuda(), attention_mask=attention_mask.cuda(), max_length=128, do_sample=True, temperature=0.1,
                               eos_token_id=tokenizer.eos_token_id, pad_token_id=tokenizer.eos_token_id)
# mostramos el resultado
print()
print(tokenizer.decode(generated_ids[0], skip_special_tokens=True))

# si nos vamos a uno más grande Salesforce/codegen2-1B, obtendremos algo mejor pero tendremos un problema similar


# Zero-shot prompting

In [ ]:
# el truco es usar un prompt similar a cómo se entrenó el modelo
PROMPT = """
# {instruction}
{response}"""

text = PROMPT.format(
    instruction="compute the mean of a numpy array",
    response=""
)
# lo tokenizamos
tokenized = tokenizer(text, return_tensors="pt")
input_ids = tokenized.input_ids
attention_mask = tokenized.attention_mask
# generamos la continuación
generated_ids = model.generate(input_ids.cuda(), attention_mask=attention_mask.cuda(), max_length=128, do_sample=True, temperature=0.1,
                               eos_token_id=tokenizer.eos_token_id, pad_token_id=tokenizer.eos_token_id)
# mostramos el resultado
print()
print(tokenizer.decode(generated_ids[0], skip_special_tokens=True))

# Few-shot prompting

In [ ]:
PROMPT = """### Instruction:
sum two numbers in python

### Response:
a + b

### Instruction:
{instruction}

### Response:
{response}"""

text = PROMPT.format(
    instruction="compute the mean of a numpy array",
    response=""
)
# lo tokenizamos
tokenized = tokenizer(text, return_tensors="pt")
input_ids = tokenized.input_ids
attention_mask = tokenized.attention_mask
# generamos la continuación
generated_ids = model.generate(input_ids.cuda(), attention_mask=attention_mask.cuda(), max_length=128, do_sample=True, temperature=0.1,
                               eos_token_id=tokenizer.eos_token_id, pad_token_id=tokenizer.eos_token_id)
# mostramos el resultado
print()
print(tokenizer.decode(generated_ids[0], skip_special_tokens=True))